# Storing HIGGS data as Parquet

- **[Overview](#overview)**
- **[Setup](#setup)**
- **[Read and Write Data to a Parquet Table in the Platform](#write-data-to-a-parquet-platform-table)**

<a id="overview"></a>
## Overview

The goal of this notebook, in addition to retrieving, processing and storing the HIGGS data set, is to sample parquet writing and reading times in an effort to determine how best to pass the data along the proposed pipeline.

**[Parquet](https://parquet.apache.org/)** is a columnar storage format that provides high-density high-performance file organization. For information about reading an writing Parquet files from Python applications, see the **[Arrow's Parquet Documentation](https://arrow.apache.org/docs/python/parquet.html)**.

<a id="setup"></a>
## Setup

Run the following code to import required libraries and ingest CSV data into a pandas DataFrame, which will be converted to a Parquet table.

In [5]:
# !pip install -U pyarrow numpy pandas

In [6]:
import os
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
import pandas as pd

#### Read remote CSV.GZ file into a pandas DataFrame and display the data and metadata that was read:

In [7]:
uci = "https://archive.ics.uci.edu/ml/machine-learning-databases"
higgs_url = "00280/HIGGS.csv.gz"

In [8]:
higgs_cols = ["labels", "lepton pT ", "lepton eta ", "lepton phi ",
              "missing energy magnitude ", "missing energy phi ", "jet 1 pt ",
              "jet 1 eta ", "jet 1 phi ", "jet 1 b-tag ", "jet 2 pt ",
              "jet 2 eta ", "jet 2 phi ", "jet 2 b-tag ", "jet 3 pt ",
              "jet 3 eta ", "jet 3 phi ", "jet 3 b-tag ", "jet 4 pt ",
              "jet 4 eta ", "jet 4 phi ", "jet 4 b-tag", "m_jj", "m_jjj",
              "m_lv ", "m_jlv", "m_bb ", "m_wbb ", "m_wwbb"]    

#### CAUTION: downloading gzips, total ~3gb, 5gb decompressed -- the following could take a while!

In [17]:
higgs = pd.read_csv(os.path.join(uci, higgs_url), header=None, names=higgs_cols)

In [18]:
# higgs = pd.read_csv('/v3io/users/admin/repos/demos/lightgbm/data/HIGGS.csv.gz', header=None, names=higgs_cols)

<a id="write-data-to-a-parquet-platform-table"></a>
## Write/Read Data to a Parquet Table in the Platform

Write the CSV data that was read into the pandas DataFrame to a Parquet table in a platform data container (i.e., in the distributed file system of the Iguazio Data Science Platform).

> **Note:** For information about using the `v3io` or `User` data mounts to reference data in the platform"s data containers, see [Platform Data Containers](collect-n-explore.ipynb/#platform-data-containers) in the **getting-started/collect-n-explore.ipynb** notebook.

Write the pandas ```DataFrame``` to a new parquet ```Table``` in the platform's file system:

In [9]:
target_path = os.path.join('/User', 'projects', 'lightgbm', 'data', 'raw')

In [6]:
%%time
pq.write_table(
    pa.Table.from_pandas(higgs),
    os.path.join(target_path, 'original'))

CPU times: user 9.71 s, sys: 622 ms, total: 10.3 s
Wall time: 13 s


Read it back again to local memory:

In [10]:
%%time
higgs = pq.read_table(os.path.join(target_path, 'original'))

CPU times: user 2.86 s, sys: 2.67 s, total: 5.53 s
Wall time: 1.85 s


In [11]:
%%time
higgs.to_pandas()

CPU times: user 1.02 s, sys: 1.76 s, total: 2.79 s
Wall time: 321 ms


,labels,lepton pT,lepton eta,lepton phi,missing energy magnitude,missing energy phi,jet 1 pt,jet 1 eta,jet 1 phi,jet 1 b-tag,...,jet 4 eta,jet 4 phi,jet 4 b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10999995,1.0,1.159912,1.013847,0.108615,1.495524,-0.537545,2.342396,-0.839740,1.320683,0.000000,...,-0.097068,1.190680,3.101961,0.822136,0.766772,1.002191,1.061233,0.837004,0.860472,0.772484
10999996,1.0,0.618388,-1.012982,1.110139,0.941023,-0.379199,1.004656,0.348535,-1.678593,2.173076,...,-0.216995,1.049177,3.101961,0.826829,0.989809,1.029104,1.199679,0.891481,0.938490,0.865269
10999997,1.0,0.700559,0.774251,1.520182,0.847112,0.211230,1.095531,0.052457,0.024553,2.173076,...,1.585235,1.713962,0.000000,0.337374,0.845208,0.987610,0.883422,1.888438,1.153766,0.931279
10999998,0.0,1.178030,0.117796,-1.276980,1.864457,-0.584370,0.998519,-1.264549,1.276333,0.000000,...,1.399515,-1.313189,0.000000,0.838842,0.882890,1.201380,0.939216,0.339705,0.759070,0.719119


### create labels

Read specific columns (attributes) from the Parquet table to save bandwidth/memory and accelerate load.

In [12]:
%%time
labels = pq.read_table(os.path.join(target_path, 'original'), columns=["labels",]).to_pandas()

CPU times: user 76.3 ms, sys: 202 ms, total: 278 ms
Wall time: 294 ms


In [13]:
%%time
# too many cols
features = pq.read_table(os.path.join(target_path, 'original'), columns=higgs_cols[1:]).to_pandas()

CPU times: user 3.67 s, sys: 5.85 s, total: 9.53 s
Wall time: 2.36 s


In [14]:
parquet_file = pq.ParquetFile(os.path.join(target_path, 'original'))
parquet_file.metadata

  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 29
  num_rows: 11000000
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 14736

### table partitions 

Since we are using parquet, what are some of the partitioning schemes that make sense for the HIGGS data? In **[An Experimental Evaluation of Large Scale GBDT Systems](https://arxiv.org/pdf/1907.01882.pdf)** the authors provide a lengthy discussion of optimal data partitioning in gradient boosting models.

There are two issues to think about when applying gradient boosting to a big data problem:

1. What is an optimal partitioning scheme for the **algorithm and its execution**.
2. What is an optimal partitioning scheme for **storage**.

The article touches on a number of important considerations concering algorithm choice with different types of big data, the dimensionality and density of the data, indexing and partitioning schemes, sparsity, caching and so on.  We won't go into details here, and strongly recommend it and the many other published research efforts on the subject.

**In summary, choosing the right algorithm from a data science perspective often has significant implications for indexing, partitioning and data storage.**

### balance

We might want to take into account whether our data is balanced or not at this early stage.  In the entire sample we are almost balanced.  We'll revisit this after we make the splits.

In [15]:
%%time
true, false = higgs.to_pandas().labels.value_counts()

CPU times: user 1.24 s, sys: 1.86 s, total: 3.1 s
Wall time: 446 ms


In [16]:
print(f'HIGGS percent True: {int(100*true/(false+true)):n}')

HIGGS percent True: 52


### prepare train-validation-test data sets

In the [kubeflow pipeline](#kubeflow%20pipeline.ipynb) notebook we will do the train-test splits during the training step, and pass the test set to the next step. In the following you can get an idea about the timings in creating the splits, writing and reading the data sets.

In [17]:
from sklearn.model_selection import train_test_split 

In [18]:
%%time
# We split using sklearn.train_trest_split twice to get a validation set:
x, xtest, y, ytest = train_test_split(features, labels, train_size=0.9, test_size=0.1)
xtrain, xvalid, ytrain, yvalid = train_test_split(x, y, train_size=0.75, test_size=0.25)

CPU times: user 12.6 s, sys: 2.5 s, total: 15.1 s
Wall time: 15.1 s


#### balance again

In [19]:
print('post-split')
print('=====================================')
true, false = ytrain.labels.value_counts()
print(f'HIGGS train percent True: {int(100*true/(false+true)):n}')
true, false = yvalid.labels.value_counts()
print(f'HIGGS valid percent True: {int(100*true/(false+true)):n}')
true, false = ytest.labels.value_counts()
print(f'HIGGS test percent True: {int(100*true/(false+true)):n}')

post-split
HIGGS train percent True: 52
HIGGS valid percent True: 53
HIGGS test percent True: 52


**time saving loading separate train/valid and test sets**

In [20]:
%%time
# FEATURES
pq.write_table(
    pa.Table.from_pandas(xtrain),
    target_path+'/xtrain')    

pq.write_table(
    pa.Table.from_pandas(xvalid),
    target_path+'/xvalid')    

pq.write_table(
    pa.Table.from_pandas(xtest),
    target_path+'/xtest')    

# LABELS
pq.write_table(
    pa.Table.from_pandas(ytrain),
    target_path+'/ytrain')    

pq.write_table(
    pa.Table.from_pandas(yvalid),
    target_path+'/yvalid')    

pq.write_table(
    pa.Table.from_pandas(ytest),
    target_path+'/ytest')

CPU times: user 11 s, sys: 1.06 s, total: 12.1 s
Wall time: 16 s


It will take about 3 seconds to load the train and test sets during the training routine, and since training is often the longest step, this delay will be relatively short and it is acceptable.

In [21]:
%%time 
xtrain = pq.read_table(target_path + '/xtrain')
ytrain = pq.read_table(target_path + '/ytrain')
xvalid = pq.read_table(target_path + '/xvalid')
yvalid = pq.read_table(target_path + '/yvalid')

CPU times: user 2.78 s, sys: 4.16 s, total: 6.95 s
Wall time: 2.62 s


Since our test set is small and loads quickly, we can load it again in the ```test_step``` routines of our pipeline.

In [22]:
%%time
xtest = pq.read_table(target_path + '/xtest')
ytest = pq.read_table(target_path + '/ytest')

CPU times: user 291 ms, sys: 317 ms, total: 608 ms
Wall time: 348 ms


### Cleanup

We won't be needing some of the files created, since they are recreated every time we run the pipeline.

In [23]:
del_list = ['xtest', 'ytest', 'xvalid', 'yvalid', 'xtrain', 'ytrain']

for file in del_list:
    tgt = os.path.join(target_path, file)
    try:
        os.remove(tgt)
    except Exception as e:
        print(f"error deleting file {tgt} {e}")

### Summary

For this particular project, apart from the initial download and conversion into a pandas DataFrame, the longest reads and writes take 2-10 seconds. We will start the pipeline with original parquet file.